# IITM RL FINAL PROJECT

## Problem - bsuite 

This notebook uses an open source reinforcement learning benchmark known as bsuite. https://github.com/deepmind/bsuite

bsuite is a collection of carefully-designed experiments that investigate core capabilities of a reinforcement learning agent.

Your task is to use any reinforcement learning techniques at your disposal to get high scores on the environments specified.

**Note**: Since the course is on Reinforcement Learning,  please limit yourself to using traditional Reinforcement Learning algorithms, 

**Do not use deep reinforcement learning.**

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

In [ ]:
!pip install -q aicrowd-cli

     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 204kB 35.0MB/s 
     |████████████████████████████████| 174kB 40.1MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


# AIcrowd Runtime Configuration 🧷

Get login API key from https://www.aicrowd.com/participants/me


In [ ]:
import os

AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
os.environ["RESULTS_DIR"] = AICROWD_RESULTS_DIR
API_KEY = ""

In [ ]:
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Install packages 🗃

Please add all pacakage installations in this section

In [ ]:
!pip install git+http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git
!pip install tabulate
!pip install tqdm

## Add any other installations you need here

  Cloning http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git to /tmp/pip-req-build-flmd8s_3
  Running command git clone -q http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git /tmp/pip-req-build-flmd8s_3
  Created wheel for bsuite: filename=bsuite-0.3.5-cp37-none-any.whl size=252042 sha256=58a583728d9d41f399ed9261ea3d17f324b7b32d1961f85769b26d760bef4bcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-pj0mvfj3/wheels/61/ea/06/77c82c07765fb8608e50e6c66bc566fa6d113c725bc6937e7b
Successfully built bsuite


# Import packages

In [ ]:
import gym
import warnings

import numpy as np
import pandas as pd
import plotnine as gg
from tqdm.notebook import tqdm

import bsuite
from bsuite.aicrowd import environments
from bsuite.aicrowd.runner import Runner
from bsuite.aicrowd.analysis import Analyzer

from scipy.stats import rv_discrete


pd.options.mode.chained_assignment = None
gg.theme_set(gg.theme_bw(base_size=16, base_family='serif'))
gg.theme_update(figure_size=(3, 1), panel_spacing_x=0.5, panel_spacing_y=0.5)
warnings.filterwarnings('ignore')

# **Agent Class**

You can modify the AGENT TEMPLATE below and implement the logic of your agent. Your agent must implement a few methods that will be called by the `Runner` class.
* `__init__` - put any initialization code here.
* `get_action` - takes in a `state` and returns an `action`.
* `learn` - takes in `(state, action, reward, next_state)`, implements the learning logic.
* `get_state` - takes in a raw `observation` directly from the env, discretizes it and returns a `state`.

In addition to these, you may implement other methods which can be called by the above methods.

Since there are multiple environments, you may need unique hyper parameters for each environment. Instantiate the agent while passing in the hyper parameters in a dictionary using the `agent_config` parameter so that each environment can use different hyper parameters for the agent while using a single `Agent` class for all of them.  You can use any names for the keys in the config dictionary.   

An example `RandomAgent` is given below.

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config=None):
        self.config = agent_config
        self.HI=self.config['high']
        self.LO=self.config['low']
        self.DISCRETE_OS_SIZE=[20]*len(self.HI)
        self.discrete_os_win_size= (np.array(self.HI)-np.array(self.LO))/self.DISCRETE_OS_SIZE
        self.learning_rate=0.1
        self.discount=0.95
        q_table= np.random.uniform(low=-2,high=0,size=(self.DISCRETE_OS_SIZE+[self.config['action_space']]))
        print(q_table.size)
        self.q_table=q_table

    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''   

        return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
        RETURNS     : 
            - NIL
        '''
        max_future_q = np.max(self.q_table[next_state] )
        current_q_value= self.q_table[state + (action,)]
        new_q = (1- self.learning_rate)* current_q_value + learning_rate * ( reward + self.discount * max_future_q)
        self.q_table[discrete_state + (action,)]= new_q


    def get_state(self, observation):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''
        
        discrete_state= (observation[0][:-1] - self.LO)/self.discrete_os_win_size
        return tuple(discrete_state.astype(np.int))
        

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# EXAMPLE
class RandomAgent:
    def __init__(self, agent_config={}):
        self.config = agent_config

    def get_action(self, state):
        action = np.random.choice(2)
        return action
    
    def learn(self, state, action, reward, next_state,done):
        if ('BAR' in self.config):
            if (self.config['BAR']):
                self.config['FOO'] += 1

    def get_state(self, observation):
        state = observation
        return state

env1_config = {
    'FOO': 0.1,
    'BAR': True
}

env2_config = {
    'FOO': 0.2,
    'BAR': False
}

randomAgent1 = RandomAgent(agent_config=env1_config)
randomAgent2 = RandomAgent(agent_config=env2_config)

# **Playing with the Environment**

#### **Instantiating the environment** :
You can create an environment by calling the following function:  
`environments.load_env(ENV_ID)` - RETURNS: `env`  
where, ENV_ID can be ONE of the following:
* `environments.CATCH`
* `environments.CATCH_NOISE`
* `environments.CARTPOLE`
* `environments.CARTPOLE_NOISE`
* `environments.MOUNTAINCAR`
* `environments.MOUNTAINCAR_NOISE`

The `NOISE` environments add a scaled random noise to the `reward`.
<br/>

#### **Runnning the environment** :
There are certain methods required to run the environments. The interface is very similar to OpenAI Gym's interfaces. Fore more information, read the OpenAI documentation [here](https://gym.openai.com/docs/).

`env.reset()` - RETURNS: `observation`  
`env.step(action)`  - RETURNS: `(next_observation, reward, done, info[NOT USED])`

There are also a few useful properties within the environments:

* `env.action_space.n` - total number of possible actions. eg: if 'n' is 3, then the possible actions are `[0, 1, 2]`
* `env.observation_space.shape` -  the shape of the observation.
* `env.bsuite_num_episodes` -  the pre-specified number of episodes which will be run during evaluation (unique for each environment).

##### *ONLY IN CATCH / CATCH_NOISE*
* `env.observation_space.high` -  the upper limit for every index in the observation.
* `env.observation_space.low` -  the lower limit for every index of the observation.
<br/>


## **Environment Observation Space Limits:**

The limits for the observation space (minimum and maximum) for all the environments are given in the table below:

| Environments                        | Limits                                                                      |
|-------------------------------------|-----------------------------------------------------------------------------|
| CATCH <br/>  CATCH_NOISE            | MIN: use `env.observation_space.low` <br/> MAX: use `env.observation_space.high` |
| CARTPOLE <br/> CARTPOLE_NOISE       | MIN: `[-1. -5., -1., -1., -5., 0.]` <br/> MAX: `[ 1.,  5.,  1.,  1.,  5., 1.]` |
| MOUNTAINCAR <br/> MOUNTAINCAR_NOISE | MIN: `[-1.2, -0.07, 0.]` <br/> MAX: `[ 0.6,  0.07,  1.]`                                 |

[NOTE] Use this code cell to play around and get used to the environments. However, the `Runner` class below will be used to evaluate your agent.

In [ ]:
env = environments.load_env(environments.MOUNTAINCAR)  # replace 'environments.CARTPOLE' with other environments

#print(env.goal_position)

Loaded bsuite_id: mountain_car/0.


In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':1,'lr_decay':10,'discount_factor':1,'learning_rate':0.1,'temp':0.1}
catch_noise_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':1,'lr_decay':10,'discount_factor':1,'learning_rate':0.1,'temp':0.1}
cartpole_config = {'env':'cartpole','state_shape':(1,20,20,1,20,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':10,'lr_decay':100,'discount_factor':1,'learning_rate':0.01,'temp':10}
cartpole_noise_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':10,'lr_decay':100,'discount_factor':1,'learning_rate':0.01}
mountaincar_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95,'learning_rate':0.1}
mountaincar_noise_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95,'learning_rate':0.1}

# Best parametrs so far
# catch_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':0.3,'lr_decay':0.3,'discount_factor':1,'learning_rate':0.1}
# catch_noise_config = {'low':[-4],'high':[4],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':0.3,'lr_decay':0.3,'discount_factor':1,'learning_rate':0.1}
# cartpole_config = {'env':'cartpole','state_shape':(1,50,50,1,50,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':0.03,'lr_decay':0.03,'discount_factor':1,'learning_rate':0.2} #0.959
# cartpole_noise_config = {'env':'cartpole','state_shape':(1,50,50,1,50,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.1, 1],'epsilon_decay':0.003,'lr_decay':0.003,'discount_factor':1,'learning_rate':0.2}
# mountaincar_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':2,'lr_decay':0.01,'discount_factor':0.95,'learning_rate':0.2}
# mountaincar_noise_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':0.03,'lr_decay':0.3,'discount_factor':0.95,'learning_rate':0.1}


In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config=None):
        self.config = agent_config
        self.q_table = np.zeros((self.config['state_shape']+(self.config['action_space'],)))
        
        self.num_ep_elapsed = 1

    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''   
        
        q_table_state = self.q_table[state]
        q_table_state = q_table_state - np.max(q_table_state)

        temp = self.config['temp']*(self.config['temp_decay']/(self.config['temp_decay']+self.num_ep_elapsed))
        softmax_param = np.exp( q_table_state/temp)
        dist = softmax_param/np.sum(softmax_param)

        action_list = (0,1,2)
        distribution = tuple(dist)
        
        random_variable = rv_discrete(values=(action_list,distribution))
        action = random_variable.rvs(size=1)[0]

        return action
    
    def learn(self, state, action, reward, next_state,done):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
        RETURNS     : 
            - NIL
        '''
        max_future_q = np.max(self.q_table[next_state] )
        current_q_value= self.q_table[state][action]
        l_r = self.get_learning_rate()
        
        self.q_table[state][action]= (1- l_r)* current_q_value + l_r * ( reward + self.config['discount_factor'] * max_future_q)
        if done :
            self.num_ep_elapsed += 1


    def get_learning_rate(self):
        return self.config['lr_decay']/(self.config['lr_decay'] + self.num_ep_elapsed)

    def get_state(self, observation):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''

        if self.config['env']=='catch':
            pos_1 = np.where(observation == 1)
            if len(np.where(observation[-1,:] == 1)[0]) == 2 :
              state = 5
            elif len(pos_1[0]) == 1 :
                state = 0
            else :
                state = pos_1[1][0] - pos_1[1][1]
            return tuple([state])

        else:
            observation = np.array(observation[0])
            discrete_win_size=(np.array(self.config["high"]) - self.config["low"])/(self.config["state_shape"]-np.ones(len(self.config["state_shape"])))
            dis_val  = (observation - self.config["low"])/discrete_win_size
            
            dis_obs = dis_val.astype(np.int)
            dis_obs=tuple(map(min,zip(self.config["state_shape"]-np.ones_like(self.config['state_shape']), tuple(map(max,zip(np.zeros_like(dis_obs), dis_obs))))))
            
            return dis_obs

        '''
        discrete_state= (observation[0][:-1] - self.LO)/self.discrete_os_win_size
        #print(discrete_state)
        return tuple(discrete_state.astype(np.int))
        '''

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# TEST AREA
env = environments.load_env(environments.CARTPOLE)  # replace 'environments.CARTPOLE' with other environments

agent = Agent(agent_config=cartpole_config )                # replace with 'Agent()' to use your custom agent

NUM_EPISODES = 1000                            # replace with 'env.bsuite_num_episodes' to run for pre-specified number of episodes
#epsilon=0.0
#start_epsilon_decaying=1
#end_epsilon_decaying=NUM_EPISODES//2
#epsilon_decay_value= NUM_EPISODES/(end_epsilon_decaying-start_epsilon_decaying)

for episode_n in tqdm(range(NUM_EPISODES)):
    done = False
    episode_reward = 0
    episode_moves = 0
 
    observation = env.reset()
    
    state = agent.get_state(observation)
   
    while not done:
        #if np.random.random() > epsilon:
        #    action = agent.get_action(state)
        #else:
        #    action= np.random.randint(0,env.action_space.n)
        action = agent.get_action(state)
        

        next_observation, reward, done, _ = env.step(action)
        
        next_state = agent.get_state(next_observation)
        #print(next_observation[-1][-1])
        
        agent.learn(state, action, reward, next_state,done)
        
        state = next_state
        

        episode_reward += reward
        episode_moves += 1
    #if start_epsilon_decaying<=episode_n<=end_epsilon_decaying:
    #    epsilon-= epsilon_decay_value
    if (((episode_n+1) % 50) == 0) : 
        print("EPISODE: ",episode_n+1,"\tREWARD: ",episode_reward,"\tEPISODE_LENGTH: ",episode_moves)
        

Loaded bsuite_id: cartpole/0.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  50 	REWARD:  124.0 	EPISODE_LENGTH:  125
EPISODE:  100 	REWARD:  111.0 	EPISODE_LENGTH:  112
EPISODE:  150 	REWARD:  524.0 	EPISODE_LENGTH:  525
EPISODE:  200 	REWARD:  299.0 	EPISODE_LENGTH:  300
EPISODE:  250 	REWARD:  313.0 	EPISODE_LENGTH:  314
EPISODE:  300 	REWARD:  131.0 	EPISODE_LENGTH:  132
EPISODE:  350 	REWARD:  86.0 	EPISODE_LENGTH:  87
EPISODE:  400 	REWARD:  93.0 	EPISODE_LENGTH:  94
EPISODE:  450 	REWARD:  100.0 	EPISODE_LENGTH:  101
EPISODE:  500 	REWARD:  118.0 	EPISODE_LENGTH:  119
EPISODE:  550 	REWARD:  190.0 	EPISODE_LENGTH:  191
EPISODE:  600 	REWARD:  105.0 	EPISODE_LENGTH:  106
EPISODE:  650 	REWARD:  96.0 	EPISODE_LENGTH:  97
EPISODE:  700 	REWARD:  93.0 	EPISODE_LENGTH:  94
EPISODE:  750 	REWARD:  92.0 	EPISODE_LENGTH:  93
EPISODE:  800 	REWARD:  128.0 	EPISODE_LENGTH:  129
EPISODE:  850 	REWARD:  92.0 	EPISODE_LENGTH:  93
EPISODE:  900 	REWARD:  83.0 	EPISODE_LENGTH:  84
EPISODE:  950 	REWARD:  452.0 	EPISODE_LENGTH:  453
EPISODE:  1000 	REWARD:  91

## Point to the Agent Class you'll use for the final score

In [ ]:
RLAgent = Agent

# **Evaluating the Agent on all the Environments**

* The following cells will take care of running your agent on each environment and aggregating the results in csv files. In each of the following cells, the `agent_config` parameter is already set to use the corresponding config dictionary for that environment. DO NOT EDIT THIS.
* Feel free to modify the `LOG_INTERVAL` parameter to change the interval between episodes for logging.  
* Please do not modify any other contents in each of the cells.  

In [ ]:
LOG_INTERVAL = 100

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_config),
    env_id = environments.CATCH,
    log_interval = LOG_INTERVAL,
)
runner.play_episodes()

Loaded bsuite_id: catch/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  1.0 	MEAN_REWARD:  0.96 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1300 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1500 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_noise_config),
    env_id = environments.CATCH_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: catch_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -0.4319180485080193 	MEAN_REWARD:  0.55 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  2.888194910660849 	MEAN_REWARD:  1.08 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  0.7058772924254098 	MEAN_REWARD:  0.97 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  0.5749511984405982 	MEAN_REWARD:  0.96 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.8165310035839968 	MEAN_REWARD:  1.07 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  0.39936420173707576 	MEAN_REWARD:  1.09 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.3884776851441103 	MEAN_REWARD:  0.91 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.8585504709523746 	MEAN_REWARD:  0.94 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.362278430457311 	MEAN_REWARD:  1.01 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.829857544425355 	MEAN_REWARD:  1.05 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -0.37396752528475785 	MEAN_REWARD:  1.03 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  0.9410664532715053 	MEAN_REWARD:  1.04 	EPISODE_LENGTH:  9


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_config),
    env_id = environments.CARTPOLE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  111.0 	MEAN_REWARD:  122.73 	EPISODE_LENGTH:  112
EPISODE:  200 	REWARD:  132.0 	MEAN_REWARD:  121.48 	EPISODE_LENGTH:  133
EPISODE:  300 	REWARD:  114.0 	MEAN_REWARD:  118.37 	EPISODE_LENGTH:  115
EPISODE:  400 	REWARD:  77.0 	MEAN_REWARD:  120.59 	EPISODE_LENGTH:  78
EPISODE:  500 	REWARD:  143.0 	MEAN_REWARD:  122.72 	EPISODE_LENGTH:  144
EPISODE:  600 	REWARD:  80.0 	MEAN_REWARD:  127.83 	EPISODE_LENGTH:  81
EPISODE:  700 	REWARD:  102.0 	MEAN_REWARD:  117.65 	EPISODE_LENGTH:  103
EPISODE:  800 	REWARD:  115.0 	MEAN_REWARD:  123.68 	EPISODE_LENGTH:  116
EPISODE:  900 	REWARD:  70.0 	MEAN_REWARD:  122.76 	EPISODE_LENGTH:  71
EPISODE:  1000 	REWARD:  72.0 	MEAN_REWARD:  128.6 	EPISODE_LENGTH:  73


In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':1,'lr_decay':10,'discount_factor':1,'learning_rate':0.1,'temp':1,'temp_decay':1}
catch_noise_config = {'low':[-4],'high':[4],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':1,'lr_decay':10,'discount_factor':1,'learning_rate':0.1}
cartpole_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':10,'lr_decay':100,'discount_factor':1,'learning_rate':0.01,'temp':1,'temp_decay':30}
cartpole_noise_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':10,'lr_decay':100,'discount_factor':1,'learning_rate':0.01}
mountaincar_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95,'learning_rate':0.1,'temp':0.1}
mountaincar_noise_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95,'learning_rate':0.1}

# Best parametrs so far
# catch_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':0.3,'lr_decay':0.3,'discount_factor':1,'learning_rate':0.1}
# catch_noise_config = {'low':[-4],'high':[4],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':0.3,'lr_decay':0.3,'discount_factor':1,'learning_rate':0.1}
# cartpole_config = {'env':'cartpole','state_shape':(1,50,50,1,50,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':0.03,'lr_decay':0.03,'discount_factor':1,'learning_rate':0.2} #0.959
# cartpole_noise_config = {'env':'cartpole','state_shape':(1,50,50,1,50,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.1, 1],'epsilon_decay':0.003,'lr_decay':0.003,'discount_factor':1,'learning_rate':0.2}
# mountaincar_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':2,'lr_decay':0.01,'discount_factor':0.95,'learning_rate':0.2}
# mountaincar_noise_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':0.03,'lr_decay':0.3,'discount_factor':0.95,'learning_rate':0.1}


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_noise_config),
    env_id = environments.CARTPOLE_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
scores_arr = []
for i in range(10) :
  runner = Runner(agent = RLAgent(agent_config=catch_config),env_id = environments.CATCH,log_interval = LOG_INTERVAL)
  runner.play_episodes()
  analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
  scores_all = analyzer.get_scores()
  scores_arr.append(scores_all['catch'])


In [ ]:
RLAgent = Agent

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_config),
    env_id = environments.MOUNTAINCAR,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -1000.0 	MEAN_REWARD:  -630.47 	EPISODE_LENGTH:  1000
EPISODE:  200 	REWARD:  -534.0 	MEAN_REWARD:  -495.34 	EPISODE_LENGTH:  534
EPISODE:  300 	REWARD:  -377.0 	MEAN_REWARD:  -507.36 	EPISODE_LENGTH:  377
EPISODE:  400 	REWARD:  -583.0 	MEAN_REWARD:  -526.69 	EPISODE_LENGTH:  583
EPISODE:  500 	REWARD:  -520.0 	MEAN_REWARD:  -521.02 	EPISODE_LENGTH:  520
EPISODE:  600 	REWARD:  -266.0 	MEAN_REWARD:  -530.95 	EPISODE_LENGTH:  266
EPISODE:  700 	REWARD:  -810.0 	MEAN_REWARD:  -697.31 	EPISODE_LENGTH:  810
EPISODE:  800 	REWARD:  -746.0 	MEAN_REWARD:  -737.01 	EPISODE_LENGTH:  746
EPISODE:  900 	REWARD:  -789.0 	MEAN_REWARD:  -679.01 	EPISODE_LENGTH:  789
EPISODE:  1000 	REWARD:  -649.0 	MEAN_REWARD:  -748.95 	EPISODE_LENGTH:  649


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_noise_config),
    env_id = environments.MOUNTAINCAR_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car_noise/1.
Logging results to CSV file for each bsuite_id in results.


EPISODE:  100 	REWARD:  -209.81047500729687 	MEAN_REWARD:  -436.41 	EPISODE_LENGTH:  207
EPISODE:  200 	REWARD:  -151.85100021473642 	MEAN_REWARD:  -296.57 	EPISODE_LENGTH:  153
EPISODE:  300 	REWARD:  -312.08191936464124 	MEAN_REWARD:  -182.95 	EPISODE_LENGTH:  313
EPISODE:  400 	REWARD:  -210.5542839170266 	MEAN_REWARD:  -167.86 	EPISODE_LENGTH:  214
EPISODE:  500 	REWARD:  -195.2194346791777 	MEAN_REWARD:  -212.67 	EPISODE_LENGTH:  194
EPISODE:  600 	REWARD:  -178.29429025818544 	MEAN_REWARD:  -167.13 	EPISODE_LENGTH:  179
EPISODE:  700 	REWARD:  -170.696077145856 	MEAN_REWARD:  -172.17 	EPISODE_LENGTH:  171
EPISODE:  800 	REWARD:  -158.98793593414734 	MEAN_REWARD:  -284.65 	EPISODE_LENGTH:  159
EPISODE:  900 	REWARD:  -154.59346953116102 	MEAN_REWARD:  -160.19 	EPISODE_LENGTH:  162
EPISODE:  1000 	REWARD:  -142.94338483942013 	MEAN_REWARD:  -156.87 	EPISODE_LENGTH:  147



# **Analysis & Result**

The following cells will show the score of the agent on each environment. The same scoring method will be used to evaluate your agent on a set of test environments.

In [ ]:
# *** PLEASE DONT EDIT THE CONTENTS OF THIS CELL ***
analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
analyzer.print_scores()

╒════════════════════╤══════════╕
│ ENVIRONMENT        │    SCORE │
╞════════════════════╪══════════╡
│ catch              │ 0.997625 │
├────────────────────┼──────────┤
│ catch_noise        │ 0.996    │
├────────────────────┼──────────┤
│ cartpole           │ 0.90162  │
├────────────────────┼──────────┤
│ cartpole_noise     │ 0.870546 │
├────────────────────┼──────────┤
│ mountain_car       │ 0.881991 │
├────────────────────┼──────────┤
│ mountain_car_noise │ 0.876081 │
╘════════════════════╧══════════╛


In [ ]:
# If you want a object to get the scores
analyzer.get_scores()

{'cartpole': 0.9016195,
 'cartpole_noise': 0.870546,
 'catch': 0.997625,
 'catch_noise': 0.996,
 'mountain_car': 0.881991,
 'mountain_car_noise': 0.876081}